In [1]:
!pip install -q optuna catboost

In [2]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import os
import pickle

import optuna
import sklearn
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import statsmodels.formula.api as smf

In [3]:
def mtest(self, x_test, y_test):
    if len(y_test.shape) == 1:
        y_test = y_test[:, None]
    y_pred = self.predict(x_test)
    diff = y_pred - y_test
    se = diff**2
    ss_res = se.sum()
    ss_total = ((y_test - self.y_train_mean)**2).sum()
    rsq = 1 - ss_res / ss_total
    rmse = np.sqrt(se.mean())
    mae = np.abs(diff).mean()
    return dict(mae=mae, rmse=rmse, rsq=rsq)

def train_with_hyp_search(x_trial, y_trial, esttype='catb', n_trials=30,
                          optuna_sql_path='/dev/shm/optuna.sqlite3',
                          optuna_storage_dir='/tmp/optuna_pkls'):

    x_trial_train, x_trial_test, y_trial_train, y_trial_test = train_test_split(x_trial, y_trial, test_size=0.1, random_state=0)
    if esttype != 'lasso':
        x_trial_train, x_trial_val, y_trial_train, y_trial_val = train_test_split(x_trial_train, y_trial_train, test_size=0.1, random_state=1)

    def objective(trial):
        successful_trials_so_far = len([t for t in study.trials if t.state.name == 'COMPLETE'])
        if successful_trials_so_far >= n_trials:
            study.stop()
            print('Maximum number of trials reached, prunning')
            raise optuna.TrialPruned()
        print(f"Running trial {trial.number}")

        if esttype == 'catb':
            param = {
                'iterations': 100_000,
                'early_stopping_rounds': trial.suggest_int("early_stopping_rounds", 50, 100),
                'verbose': 1000,
                'random_seed': successful_trials_so_far,
                #'task_type': 'GPU',
            }

            creg = CatBoostRegressor(**param)
            creg.fit(x_trial_train, y_trial_train,
                eval_set=(x_trial_val, y_trial_val)
            )

        elif esttype == 'lasso':
            alpha = trial.suggest_float("alpha", 0, 10)

            creg = sklearn.linear_model.Lasso(alpha=alpha)
            creg.fit(x_trial_train, y_trial_train)

        else:
            raise ValueError('Invalid estimator type')
        
        y_trial_pred = creg.predict(x_trial_test)
        diff = y_trial_pred - y_trial_test
        err = np.abs(diff).mean()

        trial.set_user_attr("err", err)
        print('error on hyp search validation:', err)

        best_estimator_performance = np.inf
        try:
            best_estimator_performance = study.best_trial.values[0]
        except Exception:
            pass

        if best_estimator_performance >= err:
            print('Best estimator so far. Saving estimator, started.')
            with open(f"{os.path.join(optuna_storage_dir, str(trial.number))}.pkl", "wb") as f:
                pickle.dump(creg, f)
            print('Saving estimator, done.')
        else:
            print('Not the best estimator so far.')

        print(f"Finished trial {trial.number}")
        return err

    optuna_storage_dir = os.path.join(optuna_storage_dir, esttype)
    os.makedirs(optuna_storage_dir, exist_ok=True)

    optuna_storage = optuna.storages.RDBStorage(url="sqlite:///"+optuna_sql_path, engine_kwargs={"connect_args": {"timeout": 600}})
    try:
        optuna.delete_study(storage=optuna_storage, study_name=esttype)
    except KeyError:
        pass
    study = optuna.create_study(storage=optuna_storage,
                                study_name=esttype, direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=1))
    study.optimize(objective, n_trials=n_trials)

    print("Loading best trial:", study.best_params, 'started')
    with open(f"{os.path.join(optuna_storage_dir, str(study.best_trial.number))}.pkl", "rb") as f:
        best_creg = pickle.load(f)
    print("Loading best trial:", study.best_params, 'done')

    return best_creg#, study.best_params, study

class BiasCorrectEstimatorCole:
    def __init__(self, est, x_sval_bc, y_sval_bc):
        self.est = est
        
        x_sval_bc = np.array(x_sval_bc)
        y_sval_bc = np.array(y_sval_bc).reshape(-1)
        
        reg = sklearn.linear_model.LinearRegression()
        reg.fit(y_sval_bc.reshape((-1, 1)), est.predict(x_sval_bc).reshape(-1))
        self.intercept = reg.intercept_
        self.coef = reg.coef_.item()
 
    def predict(self, x_pred, y_pred=None):
        x_pred = np.array(x_pred)
        pred = self.est.predict(x_pred)
        pred = (pred - self.intercept) / self.coef
        return pred

In [4]:
df = pd.read_csv('data/synthetic_db_healthy.csv')
df.isna().any(1).sum()

0

In [5]:
x_all, y_all = df.iloc[:, df.columns!='age_at_MRI'], df['age_at_MRI'], 
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=0)

In [6]:
best_creg  = train_with_hyp_search(x_train, y_train, esttype='catb', n_trials=30)

[I 2022-08-13 14:01:43,472] A new study created in RDB with name: catb


Running trial 0
Learning rate set to 0.003698
0:	learn: 7.2927696	test: 6.7203045	best: 6.7203045 (0)	total: 73.7ms	remaining: 2h 2m 52s
1000:	learn: 5.3641452	test: 5.4879250	best: 5.4879250 (1000)	total: 10s	remaining: 16m 28s
2000:	learn: 4.8505237	test: 5.3400868	best: 5.3400868 (2000)	total: 19.9s	remaining: 16m 12s
3000:	learn: 4.4107673	test: 5.2818156	best: 5.2815372 (2996)	total: 29.7s	remaining: 15m 58s


[I 2022-08-13 14:02:21,100] Trial 0 finished with value: 4.56010275406753 and parameters: {'early_stopping_rounds': 71}. Best is trial 0 with value: 4.56010275406753.


Stopped by overfitting detector  (71 iterations wait)

bestTest = 5.257545066
bestIteration = 3671

Shrink model to first 3672 iterations.
error on hyp search validation: 4.56010275406753
Best estimator so far. Saving estimator, started.
Saving estimator, done.
Finished trial 0
Running trial 1
Learning rate set to 0.003698
0:	learn: 7.2932588	test: 6.7211142	best: 6.7211142 (0)	total: 10.6ms	remaining: 17m 36s
1000:	learn: 5.3651469	test: 5.4913579	best: 5.4913579 (1000)	total: 10.1s	remaining: 16m 40s
2000:	learn: 4.8561194	test: 5.3463634	best: 5.3463634 (2000)	total: 20.3s	remaining: 16m 32s
3000:	learn: 4.4123265	test: 5.2897985	best: 5.2897985 (3000)	total: 30.3s	remaining: 16m 19s


[I 2022-08-13 14:02:53,648] Trial 1 finished with value: 4.5817887455261195 and parameters: {'early_stopping_rounds': 86}. Best is trial 0 with value: 4.56010275406753.


...

Stopped by overfitting detector  (87 iterations wait)

bestTest = 5.25537226
bestIteration = 4631

Shrink model to first 4632 iterations.
error on hyp search validation: 4.55663114482655
Not the best estimator so far.
Finished trial 27
Running trial 28
Learning rate set to 0.003698
0:	learn: 7.2921312	test: 6.7198612	best: 6.7198612 (0)	total: 9.79ms	remaining: 16m 18s
1000:	learn: 5.3666315	test: 5.4984025	best: 5.4984025 (1000)	total: 11.9s	remaining: 19m 32s
2000:	learn: 4.8579503	test: 5.3481235	best: 5.3481235 (2000)	total: 23.8s	remaining: 19m 27s
3000:	learn: 4.4163049	test: 5.2898636	best: 5.2891463 (2981)	total: 34.8s	remaining: 18m 45s
4000:	learn: 3.9941345	test: 5.2680504	best: 5.2678140 (3977)	total: 46.5s	remaining: 18m 35s


[I 2022-08-13 14:22:30,416] Trial 28 finished with value: 4.555813177882406 and parameters: {'early_stopping_rounds': 82}. Best is trial 26 with value: 4.5436211714457775.


Stopped by overfitting detector  (82 iterations wait)

bestTest = 5.26024738
bestIteration = 4432

Shrink model to first 4433 iterations.
error on hyp search validation: 4.555813177882406
Not the best estimator so far.
Finished trial 28
Running trial 29
Learning rate set to 0.003698
0:	learn: 7.2930631	test: 6.7207003	best: 6.7207003 (0)	total: 9.98ms	remaining: 16m 38s
1000:	learn: 5.3654081	test: 5.5015575	best: 5.5015575 (1000)	total: 12s	remaining: 19m 42s
2000:	learn: 4.8546452	test: 5.3430158	best: 5.3430049 (1999)	total: 23.1s	remaining: 18m 49s
3000:	learn: 4.4119669	test: 5.2866115	best: 5.2866115 (3000)	total: 35s	remaining: 18m 50s


[I 2022-08-13 14:23:12,248] Trial 29 finished with value: 4.5592282047466925 and parameters: {'early_stopping_rounds': 72}. Best is trial 26 with value: 4.5436211714457775.


Stopped by overfitting detector  (72 iterations wait)

bestTest = 5.274953141
bestIteration = 3494

Shrink model to first 3495 iterations.
error on hyp search validation: 4.5592282047466925
Not the best estimator so far.
Finished trial 29
Loading best trial: {'early_stopping_rounds': 92} started
Loading best trial: {'early_stopping_rounds': 92} done


In [7]:
y_pred = best_creg.predict(x_test)
err = np.abs(y_pred - y_test).mean()
print("MAE on test set", err)

MAE on test set 4.5195189067305455


In [8]:
df_rf = pd.read_csv('data/synthetic_db_with_risk_factors.csv')
x_rf = df_rf[x_all.columns]
y_rf = df_rf['age_at_MRI']
df_rf.isna().any(1).sum()

0

In [9]:
best_creg_bc = BiasCorrectEstimatorCole(best_creg, x_rf, y_rf)
y_rf_pred = best_creg_bc.predict(x_rf)

df_rf['ca_delta'] = y_rf_pred - y_rf

In [10]:
reg = smf.ols(formula='ca_delta ~ rf_diabetes*sex + age_at_MRI + age_at_MRI^2', data=df_rf).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ca_delta   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.464
Date:                Sat, 13 Aug 2022   Prob (F-statistic):           5.16e-06
Time:                        14:23:13   Log-Likelihood:            -1.2638e+05
No. Observations:               34137   AIC:                         2.528e+05
Df Residuals:                   34131   BIC:                         2.528e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1218      0.454      0.268      0.789      -0.768       1.012
rf_diabetes         0.2275      0.254      0.896      0.370      -0.270       0.725
sex                -0.5656      0.113     -5.021      0.000      -0.786      -0.345
rf_diabetes:sex    -0.0285      0.344     -0.083      0.934      -0.703       0.646
age_at_MRI         -0.0427      0.028     -1.550      0.121      -0.097       0.011
age_at_MRI ^ 2      0.0448      0.027      1.687      0.092      -0.007       0.097
==============================================================================
Omnibus:                       91.586   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               77.127
Skew:                           0.055   Prob(JB):                     1.79e-17
Kurtosis:                       2.795   Cond. No.                         782.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""